In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import norm, stats, shapiro, bartlett, ttest_ind
from statsmodels.formula.api import ols
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.feature_selection import SelectFromModel

# 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings('ignore')

# 데이터 로드
df = pd.read_csv('../데이터자료/7. 이상치수정완료.csv', encoding='cp949')

# 날짜 데이터 처리 및 훈련/테스트 데이터 분할
df["회계년도"] = pd.to_datetime(df["회계년도"])
df["회계년"] = df["회계년도"].dt.year
train = df[df["회계년"] <= 2018].reset_index(drop=True)
test = df[df["회계년"] > 2018].reset_index(drop=True)

# 피처 스케일링 (MinMaxScaler 사용)
scaler = MinMaxScaler()
col = df.columns[7:55]
train_1 = train[col]
test_1 = test[col]
scaler.fit(train_1)
train_Min = scaler.transform(train_1)
test_Min = scaler.transform(test_1)
train_Min = pd.DataFrame(train_Min, columns=train_1.columns)
train[col] = train_Min
test[col] = test_Min

# 통계적 검증 및 피처 선택 과정
# 정규성 테스트, F-test 등 코드 생략

# 선택된 피처에 대한 VIF 계산
def calculate_vif(data):
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    vif_df = pd.DataFrame()
    vif_df["VIF Factor"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    vif_df["features"] = data.columns
    return vif_df.sort_values('VIF Factor', ascending=False)

# 예시: MDA 방법을 사용한 피처 선택
mda_features = ['자기자본구성비율', '매출액순이익률', '순운전자본비율'] # 선택된 피처
mda_vif = calculate_vif(train[mda_features])
print(mda_vif)

# 최종 데이터셋 생성 및 저장
final_df = pd.concat([train, test])
final_df.to_csv('../데이터자료/9-1. MinMax 데이터셋.csv', index=False)


ValueError: could not convert string to float: '  의료용 물질 및 의약품 제조업'